In [1]:
import sqlite3
import pandas as pd

In [2]:
connect=sqlite3.connect('../data/checking-logs.sqlite', check_same_thread=False)

In [3]:
pd.io.sql.read_sql('pragma table_info(test)', connect)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


In [4]:
pd.io.sql.read_sql('select * from deadlines limit 10', connect)

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


In [5]:
pd.io.sql.read_sql('select * from test limit 10', connect)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


In [6]:
query="""select test.uid, cast((strftime('%s', test.first_commit_ts) - deadlines.deadlines)/3600 as integer) as df_min
from test left join deadlines on test.labname=deadlines.labs where not test.labname = 'project1' order by df_min asc limit 1"""

df_min = pd.io.sql.read_sql(query, connect)
df_min

,uid,df_min
0,user_30,-202


In [7]:
query="""select test.uid, cast((strftime('%s', test.first_commit_ts) - deadlines.deadlines)/3600 as integer) as df_max
from test left join deadlines on test.labname=deadlines.labs where not test.labname = 'project1' order by df_max desc limit 1"""

df_max = pd.io.sql.read_sql(query, connect)
df_max

,uid,df_max
0,user_25,-2


In [8]:
query="""select avg(cast((strftime('%s', test.first_commit_ts) - deadlines.deadlines)/3600 as integer)) as df_avg
from test left join deadlines on test.labname=deadlines.labs where not test.labname = 'project1'"""

df_avg = pd.io.sql.read_sql(query, connect)
df_avg

,df_avg
0,-89.125


In [9]:
query="""select uid, avg(differ) as avg_diff, count(uid) as pageviews from (select test.uid,
cast((strftime('%s', test.first_commit_ts) - deadlines.deadlines)/3600 as integer) as differ from test
inner join deadlines on test.labname=deadlines.labs) group by uid"""

views_diff=pd.io.sql.read_sql(query, connect)
views_diff

,uid,avg_diff,pageviews
0,user_1,-62.166667,6
1,user_10,-78.833333,6
2,user_14,-185.750000,4
3,user_17,-166.000000,6
4,user_18,-37.500000,4
5,user_19,-91.200000,5
6,user_21,-106.800000,5
7,user_25,-111.166667,6
8,user_28,-89.500000,6
9,user_3,-124.166667,6


In [10]:
views_diff.corr(numeric_only=True)

,avg_diff,pageviews
avg_diff,1.000000,0.127369
pageviews,0.127369,1.000000


In [11]:
connect.close()